In [19]:
# 신경 안 쓰셔도 됩니다.
# html 정보 가져오기 및 headers 세팅
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
            'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}

# 입출력 외에는 신경 안 쓰셔도 됩니다.
# 입력: 페이지 url의 끝부분의 page_number
# 출력: 페이지 내부에 대한 정보들을 컴퓨터가 읽기 좋은 형태
def get_url_info(page_number):
    if page_number==1:
        url = "https://www.investing.com/crypto/bitcoin/news"
    else:
        url = f"https://www.investing.com/crypto/bitcoin/news/{page_number}"
    response = requests.get(url,headers=headers)
    return BeautifulSoup(response.text,'lxml')

import requests
from bs4 import BeautifulSoup

# 입출력 외에는 신경 안 쓰셔도 됩니다.
# 입력: 
# 1. page_number: 수집할 마지막 날짜를 포함하는 페이지의 번호를 매개변수로 갖는다.
# 2. first_date: 포함시킬 날짜의 제일 과거 날짜의 전날을 'Mar 14, 2022'형식으로 입력합니다.
# 3. exculde_date: 크롤링할 첫 페이지에 원하지 않는 날짜가 포함하지 않기 위해, 포함시킬 날짜의 마지막 날짜의 바로 다음 날을 'Jun 16, 2022'형식으로 입력합니다.
# 출력:
# 뉴스 날짜 : [뉴스 기사 제목1,뉴스 기사 제목2,뉴스 기사 제목3, ... ,뉴스 기사 제목n] 형식으로 이루어진 dictionary 데이터
def get_news_data_with_date(page_number:int,first_date:str,exculde_date:str):
    date_error = '\nERROR: Date is not in proper form.\n'
    month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    if first_date[:3] not in month or exculde_date[:3] not in month:
        print(date_error)
        return {}
    elif len(first_date) != 12 or first_date[3] != ' ' or first_date[6:8] != ', ' or len(exculde_date) != 12 or exculde_date[3] != ' ' or exculde_date[6:8] != ', ':
        print(date_error)
        return {}
    
    finish=False
    news_date_title_dictionary = {}

    for page_num in range(page_number,9999999999):
        soup = get_url_info(page_num)
        news_title_list, news_date_list=[], []
        for i,title in enumerate(soup.select(".title")):
            title_string = title.get('title')
            if title_string != None:
                news_title_list.append(title_string)
            print(title_string)
        for date in soup.select(".date"):
            date_string = str(date.get_text())
            if exculde_date != date_string[3:]:
                if (first_date) == date_string[3:]:
                    finish = True
                else:
                    news_date_list.append(date_string[3:])
            
        for date, title in zip(news_date_list, news_title_list):
            if date in news_date_title_dictionary:
                news_date_title_dictionary[date].append(title)
            else:
                news_date_title_dictionary[date]=[title]
        if finish:
            break
    return news_date_title_dictionary

In [4]:
# 신경 안 쓰셔도 됩니다.
# 감정 분석에 쓸 도구 구성하기 입니다.
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 입출력 외에는 신경 안 쓰셔도 됩니다.
# 입력: string: 감정 분석하고자 하는 문장
# 출력: 감정 분석 결과 1~5 -> 클수록 긍정적인 문장
def get_sentiment(string:str):
    tokens = tokenizer.encode(string,return_tensors='pt')
    result = model(tokens)
    return (int(torch.argmax(result.logits)) + 1)

In [6]:
import pyupbit
ticker = 'KRW-BTC'
interval = 'day'
to = '2022-05-01 09:00'
count = 30
upbit_info = pyupbit.get_ohlcv(ticker=ticker,interval=interval,to=to,count=count)

In [20]:
news_date_title_dictionary = get_news_data_with_date(18,'Jun 13, 2022','Jun 16, 2022')

Oil extends gains ahead of G7 discussions on Russian exports
Ukraine and Russia: What you need to know right now
Biden urges G7 to stay together as leaders target Russian gold, oil price
A Bear Market Shopping List
S&P 500 May Fall Another 15%
1 Stock To Buy, 1 To Dump When Markets Open: Tesla, Exxon Mobil
None
None
None
None
None
None
None
None
BTC price rejects at $23K as US dollar declines from fresh 20-year highs
Crypto Flipsider News – Bitcoin Hit $22k, Three Arrows Insolvency, Celsius CEO Speaks, Tether Conspiracy, TRON Gains 15%
Banking uses 56 times more energy than Bitcoin: Valuechain report 
Soulbound Tokens: Social credit system or spark for global adoption?
To The Extreme: Crypto Fear and Greed Index Hits 7, Is it a Good Time to Buy Bitcoin?
Pre-Open Stock Movers 06/16: Large-Cap Tech Falls on Recession Fears; Twitter Gains (more...)
Bitcoin trading: Momentum strategies with different moving averages
At $20K is in “Ideal Entry Point” for Investors to Buy Bitcoin – Michael S

In [21]:
news_sentiment_analysis = pd.DataFrame({'Date':[],'News title':[],'Bitcoin price':[],'Sentiment':[]})

In [22]:
news_sentiment_analysis

,Date,News title,Bitcoin price,Sentiment


In [25]:
for i,(date,news_titles) in enumerate(news_date_title_dictionary.items()):
    for j,n_t in enumerate(news_titles):
        sentiment = get_sentiment(n_t)
        news_sentiment_analysis.loc[i+j] = [date,n_t,0,sentiment]
display(news_sentiment_analysis)

,Date,News title,Bitcoin price,Sentiment
0,"Jun 15, 2022",Oil extends gains ahead of G7 discussions on Russian exports,0,4
1,"Jun 14, 2022",Bitcoin Falls 10% In Bearish Trade,0,1
2,"Jun 14, 2022",Cardano Falls 11% In Bearish Trade,0,1
3,"Jun 14, 2022",Crypto Hedge Fund’s Tweet Fuels Speculation Over Losses,0,1
4,"Jun 14, 2022",Ethereum Falls 10% In Rout,0,1
...,...,...,...,...
41,"Jun 14, 2022","Bitcoin Off Lows but Still Under Pressure from Rate Fears, Celsius Fiasco",0,2
42,"Jun 14, 2022",Has the Time Finally Arrived for Bitcoin To Reach Its Low?,0,3
43,"Jun 14, 2022",BTC price crashes to $20.8K as ‘deadly’ candles liquidate $1.2 billion,0,1
44,"Jun 14, 2022","MicroStrategy scotches 'margin call' fears, says can withstand volatility",0,2


In [12]:
display(news_sentiment_analysis)

,Date,News title,Bitcoin price,Sentiment
